# Volcanic forcing

TODO: what do we do about Hunga Tonga and stratospheric water vapour positive forcing? What about previous eruptions?

From this paper use OMPS for 2022  https://www.nature.com/articles/s43247-022-00580-w#Sec11

In [ ]:
import glob

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from pooch import retrieve
import scipy.stats
import h5py

Download data from: https://cera-www.dkrz.de/WDCC/ui/cerasearch/entry?acronym=eVolv2k_v2 Put this in '../data/volcanic_aod/eVolv2k_v3_EVA_AOD_-500_1900_1.nc'

Download data from: https://asdc.larc.nasa.gov/project/GloSSAC/GloSSAC_2.2 Put this in '../data/volcanic_aod/GloSSAC_V2.2.nc'

Download data from: https://omps.gesdisc.eosdis.nasa.gov/data/SNPP_OMPS_Level3/OMPS_NPP_LP_L3_AER_MONTHLY.1/ Put this in '../data/volcanic_aod/SNPP_OMPS_Level3/

In [ ]:
# -500 to 1900
nc = Dataset('../data/volcanic_aod/eVolv2k_v3_EVA_AOD_-500_1900_1.nc')
aod550_evolv = nc.variables['aod550'][:]
lat_evolv = nc.variables['lat'][:]
time_evolv = nc.variables['time'][:]
nc.close()
time_evolv[-51*12]

In [ ]:
lat_evolv_bnds = np.concatenate([[90], 0.5*(lat_evolv[1:]+lat_evolv[:-1]), [-90]])
weights = -np.squeeze(np.diff(np.sin(np.radians(lat_evolv_bnds))))
aod_evolv = np.zeros((len(time_evolv)))
for i in range(len(time_evolv)):
    aod_evolv[i] = np.average(aod550_evolv[i,:],weights=weights)

In [ ]:
# 1979 to 2021
nc = Dataset('../data/volcanic_aod/GloSSAC_V2.2.nc')
data_glossac = nc.variables['Glossac_Aerosol_Optical_Depth'][:]
lat_glossac = nc.variables['lat'][:]
trp_hgt_glossac = nc.variables['trp_hgt'][:]  # lat, month
alt_glossac = nc.variables['alt'][:]
nc.close()
data_glossac[0,0,:]

In [ ]:
lat_glossac_bnds = np.concatenate(([-90], 0.5*(lat_glossac[1:]+lat_glossac[:-1]), [90]))
weights_glossac = np.diff(np.sin(np.radians(lat_glossac_bnds)))

# Glossac is at 525 nm. -2.33 Angstrom exponent from Kovilakam et al 2020, https://essd.copernicus.org/articles/12/2607/2020/essd-12-2607-2020.html
angstrom = (550/525)**(-2.33)

aod_glossac = np.zeros(516)
for i in range(516):
    aod_glossac[i] = np.average(data_glossac[i,:,2],weights=weights_glossac)*angstrom

In [ ]:
# 1850 to 2014
cmip6_file = retrieve(
    'ftp://iacftp.ethz.ch/pub_read/luo/CMIP6/CMIP_1850_2014_extinction_550nm_strat_only_v3.nc',
    known_hash='f2cd708c9cd883e6ad0cc425d7bc6820a22639a752de9563bdc48a75c2550e4c'
)

In [ ]:
nc = Dataset(cmip6_file)
ext_cmip6 = nc.variables['ext550'][:].transpose((2,1,0))  # time, height, lat
lev_cmip6 = nc.variables['altitude'][:]
lat_cmip6 = nc.variables['latitude'][:]
time_cmip6 = nc.variables['month'][:]
nc.close()

In [ ]:
time_cmip6

In [ ]:
lat_cmip6_bnds = np.concatenate(([-90], 0.5*(lat_cmip6[1:]+lat_cmip6[:-1]), [90]))
weights = np.diff(np.sin(np.radians(lat_cmip6_bnds)))
tax = np.zeros(165*12)
aod_cmip6 = np.zeros(165*12)
for i in range(0,1970,12):
    gl_mn_aod = np.average(
        np.sum(
            np.mean(ext_cmip6[i:i+12,...], axis=0) * 0.5, axis=0),
        weights=weights
    ) # 0.5 is thickness in km

for i in range(1980):
    aod_cmip6[i] = np.average(np.sum(ext_cmip6[i,...] * 0.5, axis=0), weights=weights)

In [ ]:
aod_cmip6

In [ ]:
aod_omps = np.zeros(120)

In [ ]:
angstrom = (550/510)**(-2.33)

In [ ]:
for i in range(120):
    year = (i)//12 + 2013
    month = ((i-12)%12)+1
    filename = glob.glob('../data/volcanic_aod/SNPP_OMPS_Level3/OMPS-NPP_LP-L3-AER-MONTHLY_v1.0_%4dm%02d01_*.h5' % (year, month))[0]
    h5 = h5py.File(filename)
    lat_omps = h5['/']['Latitude'][:]
    lat_omps_bnds = np.concatenate(([-90], 0.5*(lat_omps[1:]+lat_omps[:-1]), [90]))
    weights = np.diff(np.sin(np.radians(lat_omps_bnds)))
    data = h5['/']['StratColumn'][:]
    data[data==-999] = np.nan
    #data = h5['/ExtinctionAvg'][:]
    #data[data==-999] = np.nan
    #tropopause = h5['/TropopauseAltitude'][:]
    #outdata = np.zeros((24, 36))
    #for x in range(24):
    #    for y in range(36):
    #        outdata[x, y] = (np.nansum(data[x, y, int(round(tropopause[x, y])):, 0]))
    aod_omps[i] = (np.nansum(data[:,:,0] * weights*np.ones((24, 36)))/48)*angstrom
    #aod_omps[i] = (np.nansum(outdata * weights*np.ones((24, 36)))/48)*angstrom

In [ ]:
# eVolv -500 to 1849
# CMIP6 1850 to 1978
# Glossac 1979 to 2021
# OMPS 2022
aod = np.concatenate((aod_evolv[:-51*12], aod_cmip6[:129*12], aod_glossac, aod_omps[-12:]))
len(aod)

In [ ]:
# h5['/'].keys()
# data = h5['/ExtinctionAvg'][..., 0]
# h5['/Altitude'][:]
# trop = h5['/TropopauseAltitude'][:]

In [ ]:
# data[data==-999] = np.nan
# #trop[trop==-999] = np.nan

In [ ]:
# outdata = np.zeros((24, 36))
# for x in range(24):
#     for y in range(36):
#         outdata[x, y] = (np.nansum(data[x, y, int(round(trop[x, y])):]))

In [ ]:
# (np.nansum(outdata * weights*np.ones((24, 36)))/48)*angstrom

In [ ]:
# crossfade
aod[28200:28812] = (1-np.linspace(0,1,612))*aod_evolv[-51*12:]+np.linspace(0,1,612)*aod_cmip6[:612]
aod[29748:29868] = (1-np.linspace(0,1,120))*aod_cmip6[1548:1668] + np.linspace(0,1,120)*aod_glossac[:120]
aod[30180:30264] = (1-np.linspace(0,1,84))*aod_glossac[-84:] + np.linspace(0, 1, 84)*aod_omps[24:108]

In [ ]:
aod[-36:]

In [ ]:
pl.plot(aod_glossac[-108:])
pl.plot(aod_omps)
pl.plot(aod[-120:])

In [ ]:
pl.plot(np.arange(1845+1/24,1901+1/24,1/12), aod_evolv[28140:], label='Toohey & Sigl -500 to 1900 incl. background')
pl.plot(np.arange(1850+1/24,1905+1/24,1/12), aod_cmip6[:660], label='CMIP6')
pl.plot(np.arange(1845+1/24,1905+1/24,1/12), aod[28140:28860], label='blended')
pl.legend()

In [ ]:
pl.plot(np.arange(1979+1/24,2022+1/24,1/12), aod_glossac, label='Glossac')
pl.plot(np.arange(1975+1/24,2015+1/24,1/12), aod_cmip6[-480:], label='CMIP6')
pl.plot(np.arange(2013+1/24,2023+1/24,1/12), aod_omps, label='OMPS')
pl.plot(np.arange(1975+1/24,2023+1/24,1/12), aod[-576:], label='blended')
pl.legend()

In [ ]:
erf = -20 * (aod - np.mean(aod[:(2250*12)]))

In [ ]:
months = np.arange(-500+1/24,2023,1/12)
df = pd.DataFrame(data=np.vstack([aod, erf]).T, index=months, columns=['stratospheric_AOD', 'volcanic_ERF'])
df.index.name = 'year'
df.to_csv('../output/volcanic_sAOD_ERF_monthly_-50001-202212.csv')

In [ ]:
years = np.arange(-500 + 0.5, 2023)
aod_ann = np.zeros(len(aod)//12)
erf_ann = np.zeros(len(erf)//12)
for i in range(0, len(months), 12):
    aod_ann[i//12] = np.mean(aod[i:i+12])
    erf_ann[i//12] = np.mean(erf[i:i+12])
df = pd.DataFrame(data=np.vstack([aod_ann, erf_ann]).T, index=years, columns=['stratospheric_AOD', 'volcanic_ERF'])
df.index.name = 'year'
df.to_csv('../output/volcanic_sAOD_ERF_annual_-500-2022.csv')

In [ ]:
df